In [1]:
%%writefile mapper.py
#!/usr/bin/env python2
import sys
import re

total_counter = 0
stop_counter = 0

reload(sys)
sys.setdefaultencoding('utf-8') # required to convert to unicode

with open('stop_words_en.txt', 'r') as stop_file:
    stop_words_str=stop_file.read()
stop_words_list = re.split('\n', stop_words_str, flags=re.UNICODE)

for line in sys.stdin:
    try:
        article_id, text = unicode(line.strip()).split('\t', 1)
    except ValueError as e:
        continue
    words = re.split("\W*\s+\W*", text, flags=re.UNICODE)
    for word in words:
        total_counter += 1
        print >> sys.stderr, "reporter:counter:Wiki stats,Total words,%d" % 1
        if word.lower() in stop_words_list:
            stop_counter += 1
            print >> sys.stderr, "reporter:counter:Wiki stats,Stop words,%d" % 1

print "%s\t%d" % ("total", total_counter)
print "%s\t%d" % ("stop", stop_counter)

Overwriting mapper.py


In [2]:
%%writefile reducer.py
#!/usr/bin/env python2
import sys

current_key = None
word_sum = 0

for line in sys.stdin:
    try:
        key, count = line.strip().split('\t', 1)
        count = int(count)
    except ValueError as e:
        continue
    if current_key != key:
        if current_key:
            print "%s\t%d" % (current_key, word_sum)
        word_sum = 0
        current_key = key
    word_sum += count

if current_key:
    print "%s\t%d" % (current_key, word_sum)


Overwriting reducer.py


In [3]:
%%writefile reducer2.py

import sys

current_key = None
word_sum = 0
total_counter = 0
stop_counter = 0

for line in sys.stdin:
    try:
        key, count = line.strip().split('\t', 1)
        count = int(count)
    except ValueError as e:
        continue
    if current_key != key:
        if current_key:
            if current_key == 'total':
                total_counter = word_sum
            elif current_key == 'stop':
                stop_counter = word_sum
        word_sum = 0
        current_key = key
    word_sum += count

if current_key:
    if current_key == 'total':
        total_counter = word_sum
    elif current_key == 'stop':
        stop_counter = word_sum

print "%.2f" % (float((float(stop_counter) / total_counter)) * 100)

Overwriting reducer2.py


In [4]:


%%bash
INTERM_DIR="interm"
OUT_DIR="wordcount_stopwords_result"

hdfs dfs -rm -r -skipTrash ${INTERM_DIR} > /dev/null
hdfs dfs -rm -r -skipTrash ${OUT_DIR} > /dev/null

yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D mapred.job.name="Streaming wordCount stop words " \
    -D mapreduce.job.reduces=2 \
    -files mapper.py,/datasets/stop_words_en.txt,reducer.py \
    -mapper "python mapper.py" \
    -reducer "python reducer.py" \
    -input /data/wiki/en_articles_part \
    -output ${INTERM_DIR} > /dev/null
    
yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D mapred.job.name="Streaming wordCount stop words 2" \
    -D mapreduce.job.reduces=1 \
    -files reducer2.py \
    -mapper "cat" \
    -reducer "python reducer2.py" \
    -input ${INTERM_DIR}/part-0000* \
    -output ${OUT_DIR} > /dev/null    

hdfs dfs -cat ${OUT_DIR}/part-0000* | head



41.60	


rm: `wordcount_stopwords_result': No such file or directory
18/01/24 16:29:57 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
18/01/24 16:29:57 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
18/01/24 16:29:59 INFO mapred.FileInputFormat: Total input files to process : 1
18/01/24 16:30:00 INFO mapreduce.JobSubmitter: number of splits:2
18/01/24 16:30:00 INFO Configuration.deprecation: mapred.job.name is deprecated. Instead, use mapreduce.job.name
18/01/24 16:30:00 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1516806445528_0009
18/01/24 16:30:00 INFO impl.YarnClientImpl: Submitted application application_1516806445528_0009
18/01/24 16:30:00 INFO mapreduce.Job: The url to track the job: http://4d2bdd92e0f6:8088/proxy/application_1516806445528_0009/
18/01/24 16:30:00 INFO mapreduce.Job: Running job: job_1516806445528_0009
18/01/24 16:32:52 INFO mapreduce.Job: Job job_1516806445528_0009 running in uber mode : false
18/01/24 16:32:52 IN